In [31]:

import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [32]:
M1 = np.array([
    [2, 1, 3], 
    [4, 2, 1], 
    [6, 3, 4]
    ])
M2 = np.array([
    [1, 2, 3, 4], 
    [2, 4, 6, 8],
    [3, 6, 9, 12]
    ])
M3 = np.array([
    [1, 0, 2],
    [0, 1, 3],
    [0, 0, 0]
    ])

<details>
<summary><b>📘 Explanation of echelon_form </b></summary>
---

Certainly! Let's walk through the provided Python function `ef(A)` step by step. This function performs **Gaussian elimination with partial pivoting** to transform a matrix `A` into **row echelon form (REF)**, while also tracking the pivot columns.

---

### 📌 Function Signature
```python
def ef(A):
```
- Input: `A` — a 2D NumPy array (matrix).
- Output: `(U, p)` — where:
  - `U` is the upper triangular-like matrix in row echelon form.
  - `p` is a list of column indices corresponding to pivot positions.

---

### Step-by-Step Explanation

#### **1. Copy the input matrix**
```python
U = np.copy(A)
```
- Creates a copy of `A` called `U` to avoid modifying the original matrix.

#### **2. Get matrix dimensions**
```python
(m, n) = A.shape
```
- `m` = number of rows  
- `n` = number of columns

#### **3. Initialize variables**
```python
j = 0      # current column index being processed
p = []     # list to store pivot column indices
```

#### **4. Loop over each row (`i` from 0 to m−1)**
```python
for i in range(0, m):
```
- `i` is the current pivot row.

#### **5. Inner loop to find a valid pivot in column `j`**
```python
ech = 1
while (ech == 1) & (j < n):
```
- `ech` acts as a flag: stay in the loop until a non-zero pivot is found **or** we run out of columns (`j >= n`).

---

### 🔍 Inside the `while` loop:

#### **6. Find the row with the largest absolute value in column `j`, from row `i` downward**
```python
indm = np.argmax(abs(U[i:m, j]))
    # abs() will taje the abs values of the column j 
    # argmax() will return the index of the largest value 
indm = indm + i
    # since argmax() returns the index of the array, we add i to get the actual row index in U.
```
- This is **partial pivoting**: improves numerical stability by choosing the largest available pivot.

#### **7. If the current pivot element `U[i, j]` is zero (or near zero), swap rows**
```python
if (abs(U[i, j]) == 0):
    U[[i, indm], :] = U[[indm, i], :]
```
- Swaps row `i` with the row `indm` that has the largest entry in column `j`.
- Note: Even if `U[i,j]` isn't exactly zero but very small, this may still help—but the code only checks for exact zero. In practice, you'd often use a tolerance (e.g., `< 1e-12`).

#### **8. If after possible swap, the pivot is non-zero → proceed with elimination**
```python
if (abs(U[i, j]) > 0):
```

##### a. Record the pivot column
```python
p.append(j)
```

##### b. Compute multipliers for rows below the pivot
```python
M = U[i+1:m, j] / U[i, j]
```
- `M` is a vector of size `(m - i - 1,)` containing the factors to eliminate entries below the pivot.

##### c. Update the submatrix to the right of column `j`
```python
U[i+1:m, j+1:n] = U[i+1:m, j+1:n] - np.outer(M, U[i, j+1:n])
```
- `np.outer(M, U[i, j+1:n])` creates a matrix where each row is `M[k] * U[i, j+1:n]`.
- This subtracts the appropriate multiple of the pivot row from each lower row → zeros out column `j` below the pivot.

##### d. Explicitly zero out the entries below the pivot in column `j`
```python
U[i+1:m, j] = 0
```
- Although mathematically they should be zero after elimination, this ensures numerical cleanliness.

##### e. Move to next column and exit inner loop
```python
j = j + 1
ech = 0   # break out of while loop
```

#### **9. If pivot is still zero (even after swapping), move to next column**
```python
else:
    j = j + 1
    ech = 1   # continue while loop to try next column
```
- This handles cases where an entire column (from row `i` down) is zero → skip to next column.

---

### ✅ After processing all rows

#### **10. Return results**
```python
return(U, p)
```
- `U`: the matrix in (approximate) row echelon form.
- `p`: list of pivot column indices (0-based). The length of `p` equals the **rank** of the matrix (assuming no round-off errors).

---

### 🧠 Key Notes

- **Partial pivoting** is used for stability.
- The algorithm **does not normalize pivot rows** (so it's not reduced row echelon form).
- It handles **rank-deficient** or **wide/tall** matrices gracefully.
- Uses **outer product** for efficient vectorized elimination.
- Assumes exact zero comparison (`abs(U[i,j]) == 0`) — in real applications, use a tolerance like `abs(U[i,j]) < tol`.

---

### 📐 Example

If:
```python
A = [[0, 2],
     [3, 4]]
```
Then:
- Row swap occurs (since `A[0,0] == 0`)
- Pivot at column 0, then column 1
- Returns echelon form and `p = [0, 1]`

---
</details>

In [33]:
# ----------------------------------------------------------------------
# 1.1 Standard EF
# ----------------------------------------------------------------------
def ef(A): 
	U = np.copy(A) # copy the matrix A in U 
	(m,n)=A.shape
	j = 0 # index related to the column
	p =[]
	for i in range(0,m): 
		ech=1
		while (ech == 1) & (j < n):
			indm=np.argmax(abs(U[i:m,j])) # if an exchange is necessary
			indm=indm+i
			if  (abs(U[i,j])==0 ): # this row need to be changed for stability
				U[ [i, indm],:]=U[[indm,i],:]  
			if (abs(U[i,j]) > 0): 
				p.append(j)
				M = U[i+1:m,j]/U[i,j]
				U[i+1:m,j+1:n]=U[i+1:m,j+1:n]-np.outer(M,U[i,j+1:n]) 

				U[i+1:m,j]=0
				j=j+1
				ech=0
				
			else:
				j=j+1
				ech=1      
	return(U,p)  

In [34]:
# ----------------------------------------------------------------------
# 1.2 Verobose EF
# ----------------------------------------------------------------------
def ef_verbose(A, tol=1e-12):
    """
    Perform Gaussian elimination with partial pivoting (verbose mode).
    
    Parameters:
        A : 2D numpy array — input matrix
        tol : float — tolerance to treat values as zero (for numerical stability)
    
    Returns:
        U : matrix in row echelon form
        p : list of pivot column indices
    """
    print("=== Starting Gaussian Elimination with Partial Pivoting ===\n")
    U = np.copy(A).astype(float)  # Ensure floating-point for division
    m, n = U.shape
    print(f"Input matrix A ({m}x{n}):")
    print(U)
    print()

    j = 0          # current column index
    p = []         # pivot columns

    for i in range(m):
        print(f"➡️  Processing pivot row {i} (looking for pivot starting at column {j})")
        ech = 1

        while ech == 1 and j < n:
            # Find pivot row (max absolute value from row i to end in column j)
            col_segment = U[i:, j]
            ind_relative = np.argmax(np.abs(col_segment))
            indm = i + ind_relative

            current_val = U[i, j]
            max_val = U[indm, j]

            print(f"   Column {j} from row {i} downward: {col_segment}")
            print(f"   Largest entry in abs: {max_val:.3g} at row {indm}")

            # Swap if current pivot is (near) zero
            if abs(current_val) <= tol:
                if indm != i:
                    print(f"   ⚠️  Pivot U[{i},{j}] ≈ {current_val:.3g} is too small → swapping rows {i} and {indm}")
                    U[[i, indm], :] = U[[indm, i], :]
                    print(f"   Matrix after swap:")
                    print(U)
                else:
                    print(f"   ❌ All entries in column {j} from row {i} down are ~zero. Skipping column.")
            else:
                print(f"   ✅ Acceptable pivot found: U[{i},{j}] = {current_val:.3g}")

            # After possible swap, check again
            if abs(U[i, j]) > tol:
                print(f"   🎯 Using column {j} as pivot column.")
                p.append(j)

                # Compute multipliers
                if i + 1 < m:
                    M = U[i+1:m, j] / U[i, j]
                    print(f"   Multipliers (M) for rows below: {M}")

                    # Eliminate entries below pivot
                    if j + 1 < n:
                        outer_update = np.outer(M, U[i, j+1:n])
                        U[i+1:m, j+1:n] -= outer_update
                        print(f"   Updated submatrix (rows {i+1}:{m}, cols {j+1}:{n}) after elimination.")
                    else:
                        print("   No columns to the right of pivot — nothing to eliminate.")

                    # Zero out the column below pivot
                    U[i+1:m, j] = 0.0
                    print(f"   Zeroed out column {j} below row {i}.")
                else:
                    print("   No rows below pivot — elimination not needed.")

                print(f"   Matrix after processing pivot at ({i},{j}):")
                print(U)
                print()

                j += 1
                ech = 0  # exit while loop
            else:
                # Still no valid pivot — move to next column
                print(f"   ➡️  Trying next column...")
                j += 1
                ech = 1

        if j >= n:
            print(f"   🔚 Ran out of columns at row {i}. Stopping elimination.")
            break

    print("=== Gaussian Elimination Complete ===")
    print("Final matrix U (row echelon form):")
    print(U)
    print(f"Pivot columns: {p}")
    print(f"Rank estimate: {len(p)}")
    return U, p

<details>
<summary> Explanation of RREF</summary>

Absolutely! Let's walk through this `rref(A)` function **step by step**, just like we did for the previous `ef(A)` code. This function computes the **Reduced Row Echelon Form (RREF)** of a matrix using **Gaussian elimination with partial pivoting**, followed by **back substitution** to clear entries *above* each pivot and normalize pivots to 1.

---

## 📌 Function Overview

```python
def rref(A):
```
- **Input**: `A` — a 2D NumPy array (real or integer matrix).
- **Output**: `(U, p)` where:
  - `U` is the matrix in **Reduced Row Echelon Form (RREF)**.
  - `p` is a list of **pivot column indices** (0-based).

> 🔍 **RREF properties**:
> 1. All nonzero rows are above zero rows.
> 2. The leading entry (pivot) of each nonzero row is **1**.
> 3. Each pivot is the **only nonzero entry** in its column (zeros above *and* below).
> 4. Pivot in each row is to the right of the pivot in the row above.

---

## ✅ Step-by-Step Explanation

### **1. Copy input and initialize**
```python
U = np.copy(A)
(m, n) = A.shape
j = 0      # current column index
p = []     # list to store pivot columns
```
- Work on a copy so original `A` is unchanged.
- `m` = rows, `n` = columns.

---

### **2. Loop over each potential pivot row (`i = 0 to m−1`)**
```python
for i in range(0, m):
```
- `i` is the current row we want to turn into a pivot row.

---

### **3. Inner loop: find a valid pivot in column `j` or beyond**
```python
ech = 1
while (ech == 1) & (j < n):
```
- Keep searching rightward (`j++`) until we find a column with a nonzero entry from row `i` downward.

---

### **4. Partial pivoting: find row with largest absolute value in column `j`**
```python
indm = np.argmax(abs(U[i:m, j]))
indm = indm + i
```
- `indm` = index of row with largest |entry| in column `j`, from row `i` to bottom.
- This improves numerical stability.

---

### **5. Swap rows if needed**
```python
if (abs(U[indm, j]) == 0):
    U[[i, indm], :] = U[[indm, i], :]
```
> ⚠️ **Note**: This condition is **logically flawed**.  
> If `abs(U[indm, j]) == 0`, then **all entries from row `i` down in column `j` are zero** → swapping won’t help!  
> Also, the swap uses `U[indm, j]` but then checks `U[i, j]` later.  
> **Better logic**: Only swap if `U[i, j]` is zero *and* `U[indm, j]` is not.

But proceeding as written…

After this, the code checks:

---

### **6. If current pivot `U[i, j]` is nonzero → process it**
```python
if (abs(U[i, j]) > 0):
```

#### a. Record pivot column
```python
p.append(j)
```

#### b. Eliminate entries **below** the pivot (forward elimination)
```python
M = U[i+1:m, j] / U[i, j]
U[i+1:m, j+1:n] = U[i+1:m, j+1:n] - np.outer(M, U[i, j+1:n])
U[i+1:m, j] = 0
```
- Same as in `ef(A)`: zeros out column `j` below row `i`.

#### c. **Normalize the pivot row so that U[i, j] = 1**
```python
U[i, j:n] = U[i, j:n] / U[i, j]
```
- This scales the entire row from column `j` onward so the pivot becomes **exactly 1**.
- This is what distinguishes **REF** from **RREF**.

#### d. **Eliminate entries ABOVE the pivot (back substitution)**
```python
if i > 0:
    M = U[0:i, j] / U[i, j]   # But note: U[i,j] is now 1!
    U[0:i, j:n] = U[0:i, j:n] - np.outer(M, U[i, j:n])
```
- Since `U[i, j] == 1` after normalization, `M = U[0:i, j]`.
- This subtracts appropriate multiples of the pivot row from **all rows above** to zero out column `j` above the pivot.

> ✅ At this point, column `j` has:
> - Zeros everywhere except a **1 at row `i`** → full RREF condition for that column.

#### e. Move to next column and exit inner loop
```python
j = j + 1
ech = 0
```

---

### **7. If no pivot found in column `j` → skip to next column**
```python
else:
    j = j + 1
    ech = 1
```
- Happens when entire column `j` (from row `i` down) is zero.

---

### **8. Return result**
```python
return(U, p)
```
- `U`: matrix in **Reduced Row Echelon Form**.
- `p`: list of pivot columns (length = rank).

---

## ⚠️ Important Notes & Potential Issues

### 🔸 **Bug in row swap logic**
```python
if (abs(U[indm, j]) == 0): 
    U[[i, indm], :] = U[[indm, i], :]
```
- If the **maximum** absolute value in the column is zero, then **all entries are zero** → no need (and no point) to swap.
- Worse: if `U[i, j]` is already zero and `U[indm, j]` is also zero, swapping does nothing.
- **Fix suggestion**:
  ```python
  if abs(U[i, j]) < tol:  # use tolerance
      if abs(U[indm, j]) > tol:
          U[[i, indm], :] = U[[indm, i], :]
      else:
          # entire column is zero → skip
          j += 1
          continue
  ```

### 🔸 **Division by zero risk**
- If `U[i, j]` is exactly zero when normalizing, you’ll get `inf` or `nan`.
- Again, use a **tolerance** (e.g., `1e-12`) instead of `== 0`.

### 🔸 **Back substitution only if `i > 0`**
- Correct: no rows above row 0, so skip.

---

## 🧪 Example (Conceptual)

Input:
```python
A = [[2, 4, 6],
     [1, 1, 1]]
```

Steps:
1. Pivot at (0,0): scale row 0 → `[1, 2, 3]`
2. Eliminate below: row1 = row1 − 1×row0 → `[0, -1, -2]`
3. Next pivot at (1,1): scale row1 → `[0, 1, 2]`
4. Eliminate **above**: row0 = row0 − 2×row1 → `[1, 0, -1]`

Final RREF:
```
[[1, 0, -1],
 [0, 1,  2]]
```
Pivots: `p = [0, 1]`

---

## ✅ Summary: What Makes This RREF (Not Just REF)?

| Feature | Done in `rref`? |
|--------|------------------|
| Zeros below pivots | ✅ (forward elimination) |
| Pivot = 1 | ✅ (`U[i, j:n] /= U[i, j]`) |
| Zeros **above** pivots | ✅ (back substitution for `i > 0`) |
| Pivot columns strictly increasing | ✅ (`j` only increases) |

---

Would you like a **corrected version with tolerance**, or a **verbose RREF** like the one we made for `ef`?


</details>

In [35]:
# ----------------------------------------------------------------------
# 2.1 Standard RREF
# ----------------------------------------------------------------------
def rref(A):
    U = np.copy(A)
    (m,n)=A.shape
    j = 0
    p =[]
    for i in range(0,m): 
        ech=1
        while (ech == 1) & (j < n):
            indm=np.argmax(abs(U[i:m,j]))
            indm=indm+i
            if  (abs(U[indm,j]) == 0): # perform the permutation
                U[ [i, indm],:]=U[[indm,i],:] 
            if ( abs(U[i,j]) > 0):
                p.append(j)
                M=U[i+1:m,j]/U[i,j]
                U[i+1:m,j+1:n]=U[i+1:m,j+1:n]-np.outer(M,U[i,j+1:n])
                U[i+1:m,j]=0           
                U[i,j:n] = U[i,j:n]/U[i,j]   # the pivotal element should be 1        
                if i>0:
                    M= U[0:i,j]/U[i,j]
                    U[0:(i),j:n]=U[0:i,j:n]-np.outer(M,U[i,j:n])     
                    j=j+1
                    ech=0
            else:
                j=j+1
                ech=1
    return(U,p)    



In [36]:
# -----------------------------------------------------------------------
# verbose RREF
# -----------------------------------------------------------------------
def rref_verbose(A):
    """
    Verbose version of rref(A) that EXACTLY follows the logic of the provided standard RREF code,
    including the unconventional swap condition: 
        if abs(U[indm, j]) == 0: swap rows i and indm
    
    Note: This uses exact zero comparison (no tolerance) to match original behavior.
    """
    print("=== RREF (Verbose) — Exact Reproduction of Original Logic ===\n")
    U = np.copy(A).astype(float)  # ensure float for division
    m, n = U.shape
    print(f"Input matrix A ({m}×{n}):")
    print(U)
    print()

    j = 0
    p = []

    for i in range(0, m):
        if j >= n:
            print(f"➡️  Row {i}: j = {j} ≥ n = {n} → exiting loop early.")
            break

        print(f"➡️  Starting processing for pivot row i = {i}, current column j = {j}")
        ech = 1

        while (ech == 1) and (j < n):
            # Step 1: Find index of max absolute value in column j from row i down
            ind_relative = np.argmax(abs(U[i:m, j]))
            indm = ind_relative + i
            print(f"   → Column {j}, rows {i} to {m-1}: {U[i:m, j]}")
            print(f"   → argmax(abs(...)) = {ind_relative} → absolute row index indm = {indm}")

            # Step 2: UNCONVENTIONAL SWAP CONDITION (as in original code)
            if abs(U[indm, j]) == 0:
                print(f"   🔄 Condition met: abs(U[{indm},{j}]) == 0 → swapping rows {i} and {indm}")
                U[[i, indm], :] = U[[indm, i], :]
                print(f"   Matrix after swap:")
                print(U)
            else:
                print(f"   ➖ No swap: abs(U[{indm},{j}]) = {U[indm, j]} ≠ 0")

            # Step 3: Check if current pivot (U[i,j]) is non-zero
            if abs(U[i, j]) > 0:
                print(f"   ✅ Pivot found at ({i},{j}) = {U[i, j]}")
                p.append(j)
                print(f"   → Recording column {j} as pivot column. Current pivots: {p}")

                # Forward elimination (below pivot)
                if i + 1 < m:
                    M = U[i+1:m, j] / U[i, j]
                    print(f"   → Multipliers for forward elimination (rows below): {M}")
                    if j + 1 < n:
                        U[i+1:m, j+1:n] = U[i+1:m, j+1:n] - np.outer(M, U[i, j+1:n])
                    U[i+1:m, j] = 0
                    print(f"   → Zeroed column {j} below row {i}")
                else:
                    print("   → No rows below pivot — forward elimination skipped")

                # Normalize pivot row so U[i,j] = 1
                U[i, j:n] = U[i, j:n] / U[i, j]
                print(f"   ➗ Normalized row {i} so pivot = 1")
                print(f"   Matrix after normalization:")
                print(U)

                # Back substitution (above pivot)
                if i > 0:
                    M = U[0:i, j] / U[i, j]  # note: U[i,j] is now 1
                    print(f"   → Multipliers for back substitution (rows above): {M}")
                    if j + 1 < n:
                        U[0:i, j+1:n] = U[0:i, j+1:n] - np.outer(M, U[i, j+1:n])
                    U[0:i, j] = 0
                    print(f"   → Zeroed column {j} above row {i}")
                    print(f"   Matrix after back substitution:")
                    print(U)
                else:
                    print("   → No rows above pivot — back substitution skipped")

                j = j + 1
                ech = 0
                print(f"   → Moving to next column: j = {j}\n")

            else:
                print(f"   ❌ No pivot at ({i},{j}) → abs(U[{i},{j}]) = 0")
                j = j + 1
                ech = 1
                print(f"   → Trying next column: j = {j}\n")

        if j >= n:
            print(f"   🔚 j = {j} ≥ n = {n} → breaking out of while loop\n")

    print("=== Final Result ===")
    print("RREF matrix U:")
    print(U)
    print(f"Pivot columns: {p}")
    return U, p

In [37]:
rref_verbose(M1)


=== RREF (Verbose) — Exact Reproduction of Original Logic ===

Input matrix A (3×3):
[[2. 1. 3.]
 [4. 2. 1.]
 [6. 3. 4.]]

➡️  Starting processing for pivot row i = 0, current column j = 0
   → Column 0, rows 0 to 2: [2. 4. 6.]
   → argmax(abs(...)) = 2 → absolute row index indm = 2
   ➖ No swap: abs(U[2,0]) = 6.0 ≠ 0
   ✅ Pivot found at (0,0) = 2.0
   → Recording column 0 as pivot column. Current pivots: [0]
   → Multipliers for forward elimination (rows below): [2. 3.]
   → Zeroed column 0 below row 0
   ➗ Normalized row 0 so pivot = 1
   Matrix after normalization:
[[ 1.   0.5  1.5]
 [ 0.   0.  -5. ]
 [ 0.   0.  -5. ]]
   → No rows above pivot — back substitution skipped
   → Moving to next column: j = 1

➡️  Starting processing for pivot row i = 1, current column j = 1
   → Column 1, rows 1 to 2: [0. 0.]
   → argmax(abs(...)) = 0 → absolute row index indm = 1
   🔄 Condition met: abs(U[1,1]) == 0 → swapping rows 1 and 1
   Matrix after swap:
[[ 1.   0.5  1.5]
 [ 0.   0.  -5. ]
 [ 0.

(array([[1. , 0.5, 0. ],
        [0. , 0. , 1. ],
        [0. , 0. , 0. ]]),
 [0, 2])